In [23]:

%run closest_versus_minimal_tools.ipynb


In [2]:
import matplotlib.pyplot as plt


In [25]:
# # This part of code is meant to generate the precise points the algorithm took during the descent. 


# import matplotlib.pyplot as plt



# def plot_pathway(pathway, result):
#     '''
#     Plots the pathway of the descent algorithm in following manner:

#     red dot - lcCube
#     blue small dots - pathway of the descent
#     blue big dot - end of the descent, if not seen the descent ended successfully
#     orange big dot - lcLLL
#     '''
#     diff_indices = different_components(pathway)

#     x_pathway = [point[diff_indices[0]] for point in pathway]
#     y_pathway = [point[diff_indices[1]] for point in pathway]
    
#     # Create the plot
#     plt.scatter(x_pathway[-1],y_pathway[-1], color='orange',s=70)  # Highlight the last point lcLLL in green
#     plt.scatter(x_pathway[1:-1],y_pathway[1:-1], color='blue',s=10)
#     plt.scatter(x_pathway[0],y_pathway[0], color='red')  # Highlight the first point lcCUBE in red

#     # Add description next to the red point
#     description = f"({x_pathway[0]:.2f}, {y[0]:.2f})"
#     plt.text(x_pathway[0] + 0.1,y_pathway[0] - 0.1, description)

#     # Add descriptions to the blue points
#     for i in range(1, len(x) - 1):
#         description = f"({x_pathway[i]:.2f}, {y[i]:.2f})"
#         plt.text(x_pathway[i] + 0.1,y_pathway[i] - 0.1, description)

#     # Add gridlines corresponding to integers
#     plt.grid(True, which='both')

#     # Set axis ticks to integers only
#     plt.xticks(range(int(min(x)) - 1, int(max(x)) + 2))
#     plt.yticks(range(int(min(y)) - 1, int(max(y)) + 2))

#     # Show the plot
#     print("plot_pathway: reached LLL?", result)
    
#     plt.show()

In [27]:



def generate_nearby_integer_points(lcCube, exception_index, dist):
    '''
    Generate nearby integer points of lcCube within a given distance, excluding a specific index.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        exception_index (int): The index of the coordinate to exclude from the calculation.
        dist (int): The maximum distance to generate nearby integer points.
    '''
    if dist<2:
        return ValueError("Neighborhood insufficiently large.")

    n = len(lcCube)
    nearby_points = []
    
    # Calculate the range of possible integer coordinates for each dimension, except at index ind
    ranges = [range(int(round(coord) - dist + 1), int(round(coord) + dist)) if i != exception_index else [lcCube[exception_index]] for i, coord in enumerate(lcCube)]

    # Generate all possible combinations of integer coordinates within the ranges
    coordinate_combinations = list(itertools.product(*ranges))

    for point in coordinate_combinations:
        total_diff = sum(abs(lcCube[i] - point[i]) for i in range(n))
        if total_diff < dist:
            nearby_points.append(list(point))
    
    return nearby_points


def remove_coordinate(index, points, lcCube, lcLLL):
    '''
    Remove a coordinate from a list of points, lcCube, and lcLLL at the specified index.
    '''
    filtered_points = [remove_coordinate_single(index, point) for point in points]
    lcCubeRemoved = remove_coordinate_single(index, lcCube)
    lcLLLRemoved = remove_coordinate_single(index, lcLLL)
    return filtered_points, lcCubeRemoved, lcLLLRemoved


def remove_coordinate_single(index, point):
    '''
    Remove a coordinate from a single point at the specified index.
    INPUTS:
        index (int): The index of the coordinate to be removed.
        point (list): The point represented as a list of coordinates.
    '''
    point_removed = point[:]
    point_removed.pop(index)
    return point_removed

In [31]:
from math import floor, ceil

def small_cube(floats):
    n = len(floats)
    seen_combinations = set()  
    for i in range(2**n):
        combination = []
        for j in range(n):
            if i & (1 << j):
                combination.append(ceil(floats[j]))
            else:
                combination.append(floor(floats[j]))
        seen_combinations.add(tuple(combination))  # Convert list to tuple for hashability
    return [list(comb) for comb in seen_combinations]

# Example
# floats = [1.4, 5.6]
# unique_roundings = generate_roundings_no_duplicates(floats)
# print(unique_roundings)



# # # # Example usage
# # result = generate_roundings_no_duplicates([3.0, -10.320968627929688, 12.099395751953125])
# print(result)




[[1, 6], [2, 5], [2, 6], [1, 5]]
[[3, -11, 12], [3, -10, 12], [3, -11, 13], [3, -10, 13]]


In [37]:
def generate_plus_minus_ones(numbers):
    """
    Generates all combinations of the given numbers with +-1 applied to each entry.

    Args:
    - numbers: List[int] - list of integers.

    Returns:
    - List[List[int]] - list of lists, each representing one combination.
    """
    result = []
    for i, num in enumerate(numbers):
        # create copies of the original list with +1 and -1 applied to the current number
        plus_one = numbers[:i] + [num + 1] + numbers[i+1:]
        minus_one = numbers[:i] + [num - 1] + numbers[i+1:]
        result.extend([plus_one, minus_one])
    return result

# example usage
print(generate_plus_minus_ones([1, 5]))


[[2, 5], [0, 5], [1, 6], [1, 4]]
